In [30]:
%load_ext dotenv
%dotenv

In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from dotenv import load_dotenv
from langchain.document_loaders import JSONLoader

In [25]:
import os
import openai
from dotenv import load_dotenv

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = ""
os.environ["OPENAI_API_KEY"] = ""
os.environ["DEFAULT_EMBED_BATCH_SIZE"] = "10"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate

In [5]:
llm = OpenAI(engine="davinci",temperature=0)
text_splitter = CharacterTextSplitter()

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [18]:
from langchain.document_loaders import JSONLoader
import logging, json, os
from langchain.text_splitter import RecursiveCharacterTextSplitter
loader = JSONLoader("data/MedicalRecords.json", jq_schema='.[]', text_content=False)
rawDocs = loader.load()

textSplitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=0)
docs = textSplitter.split_documents(rawDocs)
logging.info("Docs " + str(len(docs)))

In [19]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

In [46]:
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [21]:

from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="gpt-35-turbo")

In [22]:
from langchain.chains.summarize import load_summarize_chain

In [23]:
from langchain.chat_models import ChatOpenAI
llm=ChatOpenAI(temperature=0.7, engine="gpt-4")
# llm=ChatOpenAI(temperature=0.7, engine="gpt-4", max_tokens=1000)

                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [30]:
prompt_template = """You are an AI assistant tasked with summarizing medical records documents. Your summary should accurately capture the key information in the document while avoiding the omission of any domain-specific words. Please generate a concise and comprehensive summary that includes details such as patient information, medical history, allergies, chronic conditions, previous surgeries, prescribed medications, and upcoming appointments. Ensure that the summary is easy to understand for healthcare professionals and provides an accurate representation of the patient's medical history and current health status. Begin the summary with a brief introduction of the patient, followed by the main points of their medical records. Please remember to use clear language and maintain the integrity of the original information without missing any important details:


{text}

"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
chain = load_summarize_chain(llm, chain_type="map_reduce", return_intermediate_steps=False, map_prompt=PROMPT, combine_prompt=PROMPT)
# chain = load_summarize_chain(llm, chain_type="refine", return_intermediate_steps=True)
chain({"input_documents": docs}, return_only_outputs=True)

{'output_text': 'John Doe, born May 15, 1980, suffers from lower back pain, hypertension, and type 2 diabetes. He is allergic to penicillin and peanuts and has a history of appendectomy and knee arthroscopy. His vitals are stable, but he has diminished reflexes at the knees and ankles. Diagnosed with low back pain, hypertension, and diabetes, he is prescribed Ibuprofen and Metformin, physical therapy, and referred to Orthopedics for further evaluation.'}